# Install Spark

In [0]:
# Install latest version of spark. If error, check the latest and replace "spark-2.4.4"
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://www-us.apache.org/dist/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz
!tar xf spark-2.4.4-bin-hadoop2.7.tgz
!pip install -q findspark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.4-bin-hadoop2.7"
import findspark
findspark.init()

In [2]:
import numpy as np
import pandas as pd

from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


# Upload kmeans.py provided

In [0]:
import operator
import sys
from pyspark import SparkConf, SparkContext
import numpy as np
import matplotlib.pyplot as plt
from scipy import linalg

In [0]:
# Macros.
MAX_ITER = 20
DATA_PATH = "/content/gdrive/My Drive/BigData/q1/data.txt"
C1_PATH = "/content/gdrive/My Drive/BigData/q1/c1.txt"
C2_PATH = "/content/gdrive/My Drive/BigData/q1/c2.txt"
NORM = 2

In [0]:
# Load data (corresponding to the def main())
# Spark settings
conf = SparkConf()
sc = SparkContext(conf=conf)

# Load the data, cache this since we're accessing this each iteration
data = sc.textFile(DATA_PATH).map(
        lambda line: np.array([float(x) for x in line.split(' ')])
        ).cache()
# Load the initial centroids c1, split into a list of np arrays
centroids1 = sc.textFile(C1_PATH).map(
        lambda line: np.array([float(x) for x in line.split(' ')])
        ).collect()
# Load the initial centroids c2, split into a list of np arrays
centroids2 = sc.textFile(C2_PATH).map(
        lambda line: np.array([float(x) for x in line.split(' ')])
        ).collect()

In [0]:
# Helper functions.
def closest(p, centroids, norm):
    """
    Compute closest centroid for a given point.
    Args:
        p (numpy.ndarray): input point
        centroids (list): A list of centroids points
        norm (int): 1 or 2
    Returns:
        int: The index of closest centroid.
    """
    closest_c = min([(i, linalg.norm(p - c, norm))
                    for i, c in enumerate(centroids)],
                    key=operator.itemgetter(1))[0]
    return closest_c

In [0]:
# K-means clustering
def kmeans(data, centroids, norm=2):
    """
    Conduct k-means clustering given data and centroid.
    This is the basic version of k-means, you might need more
    code to record cluster assignment to plot TSNE, and more
    data structure to record cost.
    Args:
        data (RDD): RDD of points
        centroids (list): A list of centroids points
        norm (int): 1 or 2
    Returns:
        RDD: assignment information of points, a RDD of (centroid, (point, 1))
        list: a list of centroids
        and define yourself...
    """
    # iterative k-means
    for _ in range(MAX_ITER):
        # Transform each point to a combo of point, closest centroid, count=1
        # point -> (closest_centroid, (point, 1))
        data_trans = data.map(lambda p:(closest(p, centroids,norm),(p,1)))
        
        # Re-compute cluster center
        # For each cluster center (key), aggregate its values
        # by summing up points and count
        clusters = data_trans.reduceByKey(lambda p1_c, p2_c: (p1_c[0]+p2_c[0], p1_c[1]+p2_c[1]))
        
        # Average the points for each centroid: divide sum of points by count
        # Use collect() to turn RDD into list
        centroids = clusters.map(lambda c:(c[0], c[1][0]/c[1][1])).collect()
        
    data_trans = data.map(lambda p:(closest(p, centroids,norm),(p,1)))
    return data_trans, centroids

In [0]:
centroids = centroids1
norm = NORM
for _ in range(MAX_ITER):
    # Transform each point to a combo of point, closest centroid, count=1
    # point -> (closest_centroid, (point, 1))
    data_trans = data.map(lambda p:(closest(p, centroids,norm),(p,1)))

    # Re-compute cluster center
    # For each cluster center (key), aggregate its values
    # by summing up points and count
    clusters = data_trans.reduceByKey(lambda p1_c, p2_c: (p1_c[0]+p2_c[0], p1_c[1]+p2_c[1]))

    # Average the points for each centroid: divide sum of points by count
    # Use collect() to turn RDD into list
    centroids = clusters.map(lambda c:(c[0], c[1][0]/c[1][1])).collect()

In [0]:
data_trans, centroids = kmeans(data, centroids1)

In [0]:
from pyspark.mllib.clustering import KMeans
clusters = KMeans.train(data, 10, maxIterations=20, initializationMode="random")

# 自己写

In [15]:
closest(data.collect()[50], centroids1, NORM)

7

In [0]:
(closest_centroid, (point, 1))

In [0]:
data_trans = data.map(lambda p:(closest(p, centroids1,NORM),(p,1)))

In [0]:
clusters = data_trans.reduceByKey(lambda p1_c, p2_c: (p1_c[0]+p2_c[0], p1_c[1]+p2_c[1]))

In [0]:
new_centroids = clusters.map(lambda c:(c[0], c[1][0]/c[1][1])).collect()

In [42]:
type(new_centroids),np.shape(new_centroids), np.shape(centroids1)

(list, (10, 2), (10, 58))

# 1. Run clustering on data.txt with c1.txt and c2.txt as initial centroids and use L1
distance as similarity measurement. Compute and plot the within-cluster cost for
each iteration. You’ll need to submit two graphs here.

In [0]:
c1 = spark.read.option("delimiter", " ").option("header", "false").csv('/content/gdrive/My Drive/BigData/q1/c1.txt')
print(c1.count(),len(c1.columns))
#c1.show(5)
c1_pd = c1.toPandas().astype('float')
#c1_pd.head()

10 58


In [0]:
c2 = spark.read.option("delimiter", " ").option("header", "false").csv('/content/gdrive/My Drive/BigData/q1/c2.txt')
print(c2.count(),len(c2.columns))
c2_pd = c2.toPandas().astype('float')
#c2_pd.head()

10 58


In [0]:
data = spark.read.option("delimiter", " ").option("header", "false").csv('/content/gdrive/My Drive/BigData/q1/data.txt')
print(data.count(),len(data.columns))
data_pd = data.toPandas().astype('float')
#data_pd.head()

4601 58


In [0]:
npoint, ndim = np.shape(data_pd)

In [0]:
def kMeans(initial, norm, k, max_iter):
  for _ in range(max_iter):
    for point in 

In [0]:
def distance(pointA, pointB, norm):
  dist = 0
  if norm == 'L1':
    dist = sum(abs(pointA.sub(pointB, fill_value=0)))
  elif norm == 'L2':
    dist = sum(pointA.sub(pointB, fill_value=0)**2)
  return dist

def find_closest(point, centroids, norm):
  min_dist, closest_index = 0, 0
  for i in range(k):
    dist = distance(point, centroids.iloc[i], norm)
    #print(i, min_dist, dist)
    if dist < min_dist:
      min_dist = dist
      closest_index = i
      #print(closest_index)
  return closest_index

In [0]:
find_closest(data_pd.iloc[0], centroids, norm)

0 0 0.0
1 0 800.234
2 0 2420.864
3 0 116.61699999999999
4 0 116.61699999999999
5 0 281.557
6 0 234.777
7 0 291.15
8 0 1381.691
9 0 495.992


0

In [0]:
k = 10
max_iter = 20
norm = 'L1'
centroids = c1_pd

for _ in range(max_iter):
  # assign the closest centroid to each point
  cluster_ids = dict()
  for i in range(npoint):
    cluster_id = find_closest(data_pd.iloc[i], centroids, norm)
    #print(cluster_id)
    cluster_ids[cluster_id] = cluster_ids.get(cluster_id, []) + [i]
    #print(cluster_ids[cluster_id])  
  # re-compute cluster center
  for i in range(k):
    cluster_i = data_pd.loc[cluster_ids[i]]
    centroid_i = cluster_i.mean(axis=0)
    centroids.loc[i] = centroid_i

0
[0]
0
[0, 1]
0
[0, 1, 2]
0
[0, 1, 2, 3]
0
[0, 1, 2, 3, 4]
0
[0, 1, 2, 3, 4, 5]
0
[0, 1, 2, 3, 4, 5, 6]
0
[0, 1, 2, 3, 4, 5, 6, 7]
0
[0, 1, 2, 3, 4, 5, 6, 7, 8]
0
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
0
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
0
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
0
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
0
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]
0
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]
0
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
0
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]
0
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17]
0
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]
0
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
0
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
0
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21]
0
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22]
0
[0, 1, 2, 

KeyboardInterrupt: ignored

In [0]:
cluster_ids = dict()
cluster_ids[0] = cluster_ids.get(0,[]) + [0]
cluster_ids[0]

[0]

In [0]:
test = dict()
test[0] = test.get(0,[]) + [0]
test[0]

[0]

In [0]:
test

{0: None}

In [0]:
data_pd.mean(axis=0)

_c0       0.104553
_c1       0.213015
_c2       0.280656
_c3       0.065425
_c4       0.312223
_c5       0.095901
_c6       0.114208
_c7       0.105295
_c8       0.090067
_c9       0.239413
_c10      0.059824
_c11      0.541702
_c12      0.093930
_c13      0.058626
_c14      0.049205
_c15      0.248848
_c16      0.142586
_c17      0.184745
_c18      1.662100
_c19      0.085577
_c20      0.809761
_c21      0.121202
_c22      0.101645
_c23      0.094269
_c24      0.549504
_c25      0.265384
_c26      0.767305
_c27      0.124845
_c28      0.098915
_c29      0.102852
_c30      0.064753
_c31      0.047048
_c32      0.097229
_c33      0.047835
_c34      0.105412
_c35      0.097477
_c36      0.136953
_c37      0.013201
_c38      0.078629
_c39      0.064834
_c40      0.043667
_c41      0.132339
_c42      0.046099
_c43      0.079196
_c44      0.301224
_c45      0.179824
_c46      0.005444
_c47      0.031869
_c48      0.038575
_c49      0.139030
_c50      0.016976
_c51      0.269071
_c52      0.

# 2. Run clustering on data.txt with c1.txt and c2.txt as initial centroids and use L2
distance as similarity measurement. Compute and plot the within-cluster cost for
each iteration. You’ll need to submit two graphs here.

# 3. T-SNE is a dimensionality reduction method particularly suitable for visualization
of high-dimensional data. Visualize your clustering assignment result of (2) by
reducing the dimension to a 2D space. You’ll need to submit two graphs here.

# 4. For L2 and L1 distance, are random initialization of K-means using c1.txt better
than initialization using c2.txt in terms of cost? Explain your reasoning.